In [1]:
import json

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp

import matplotlib.pyplot as plt

from pepbench.io import load_challenge_results_from_folder
from pepbench.io import convert_hz_to_ms
from pepbench.datasets import EmpkinsDataset, GuardianDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
root_path = Path("../../")

In [3]:
result_path = root_path.joinpath("results")
result_path

WindowsPath('../../results')

In [4]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(empkins_base_path)

\Users\sebas\Development\ResearchInternship\Data\2024_08_PEP_Benchmarking\EmpkinS_Dataset


#### Set flags for further processing

In [5]:
save_results = True
include_rr_interval = False

In [6]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
fs_empkins = dataset_empkins.sampling_rate_icg
print(f"Sampling rate ICG: {fs_empkins}")
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
fs_guardian = dataset_guardian.sampling_rate_icg
print(f"Sampling rate ICG: {fs_guardian}")

Sampling rate ICG: 1000
Sampling rate ICG: 500


In [7]:
algo_levels = ["q_wave_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]
algo_level_mapping = dict(zip(algo_levels, ["Q-Wave Algorithm", "B-Point Algorithm", "Outlier Correction"]))

#### Load data from the challenge results

In [8]:
results_empkins = load_challenge_results_from_folder(
    result_path.joinpath("empkins_dataset_reference_b_point").resolve(), index_cols_per_sample=["participant", "condition", "phase"]
).per_sample
results_guardian = load_challenge_results_from_folder(
    result_path.joinpath("guardian_dataset_reference_b_point").resolve(), index_cols_per_sample=["participant", "phase"]
).per_sample

In [9]:
results_empkins = results_empkins.reset_index().set_index(['q_wave_algorithm', 'b_point_algorithm', 'outlier_correction_algorithm', 'participant', 'condition', 'phase'])
results_empkins = results_empkins.drop(columns=['level_6'])
results_empkins

C:\Users\sebas\AppData\Local\Temp\ipykernel_16580\1857737345.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  results_empkins = results_empkins.drop(columns=['level_6'])


heartbeat_id  \
                                                                                                  estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                  
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep               0   
                                                                                       Prep               1   
                                                                                       Prep               2   
                                                                                       Prep               3   
                                                                                       Prep               4   
...                                                                                                     ...   
vanlien2013-42-ms b-point-reference none                         VP_032      ftsst     Pause_5            7   
                                                                                       Pause_5            8   
                                                                                       Pause_5            9   
                                                                                       Pause_5           10   
                                                                                       Pause_5           11   

                                                                                                          \
                                                                                               reference   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase               
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep            0   
                                                                                       Prep            1   
                                                                                       Prep            2   
                                                                                       Prep            3   
                                                                                       Prep            4   
...                                                                                                  ...   
vanlien2013-42-ms b-point-reference none                         VP_032      ftsst     Pause_5         7   
                                                                                       Pause_5         8   
                                                                                       Pause_5         9   
                                                                                       Pause_5        10   
                                                                                       Pause_5        11   

                                                                                               heartbeat_start_sample  \
                                                                                                            estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                            
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep                       394   
                                                                                       Prep                       984   
                                                                                       Prep                      1569   
                                                                                       Prep                      2134   
                                                                                       Prep                      2684   
...                                                        

#### Introduce column 'condition' to the guardian dataset to assure compatibility with the empkins dataset

In [10]:
results_guardian = results_guardian.assign(condition="Dummy")
results_guardian = results_guardian.reset_index().set_index(['q_wave_algorithm', 'b_point_algorithm', 'outlier_correction_algorithm', 'participant', 'condition', 'phase']).drop(columns=['level_5'])
results_guardian

C:\Users\sebas\AppData\Local\Temp\ipykernel_16580\1754320087.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  results_guardian = results_guardian.reset_index().set_index(['q_wave_algorithm', 'b_point_algorithm', 'outlier_correction_algorithm', 'participant', 'condition', 'phase']).drop(columns=['level_5'])


heartbeat_id  \
                                                                                                   estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                   
forounzafar2018   b-point-reference none                         GDN0005     Dummy     Pause               0   
                                                                                       Pause               1   
                                                                                       Pause               2   
                                                                                       Pause               3   
                                                                                       Pause               4   
...                                                                                                      ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     Dummy     TiltDown           51   
                                                                                       TiltDown           52   
                                                                                       TiltDown           53   
                                                                                       TiltDown           54   
                                                                                       TiltDown           55   

                                                                                                           \
                                                                                                reference   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                
forounzafar2018   b-point-reference none                         GDN0005     Dummy     Pause            0   
                                                                                       Pause            1   
                                                                                       Pause            2   
                                                                                       Pause            3   
                                                                                       Pause            4   
...                                                                                                   ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     Dummy     TiltDown        51   
                                                                                       TiltDown        52   
                                                                                       TiltDown        53   
                                                                                       TiltDown        54   
                                                                                       TiltDown        55   

                                                                                                heartbeat_start_sample  \
                                                                                                             estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                             
forounzafar2018   b-point-reference none                         GDN0005     Dummy     Pause                       352   
                                                                                       Pause                       808   
                                                                                       Pause                      1252   
                                                                                       Pause                      1686   
                                                                                       Pause                      2116   
...                     

#### Perform data preprocessing

In [11]:
results_empkins_norm = results_empkins.copy()
results_guardian_norm = results_guardian.copy()
results_empkins_norm[("q_wave_onset_sample", "estimated")] = results_empkins[("q_wave_onset_sample", "estimated")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_empkins_norm[("q_wave_onset_sample", "reference")] = results_empkins[("q_wave_onset_sample", "reference")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("q_wave_onset_sample", "estimated")] = results_guardian[("q_wave_onset_sample", "estimated")] - results_guardian[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("q_wave_onset_sample", "reference")] = results_guardian[("q_wave_onset_sample", "reference")] - results_guardian[("heartbeat_start_sample", "estimated")]

In [12]:
cols = [("heartbeat_start_sample", "estimated"), ("heartbeat_start_sample", "reference"), ("heartbeat_end_sample", "estimated"), ("heartbeat_end_sample", "reference"), ("q_wave_onset_sample", "estimated"), ("q_wave_onset_sample", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference"), ("pep_sample", "estimated"), ("pep_sample", "reference")]
results_empkins_cleaned_ms = results_empkins_norm.copy()
results_guardian_cleaned_ms = results_guardian_norm.copy()
results_empkins_cleaned_ms[cols] = results_empkins_norm[cols] * convert_hz_to_ms(sampling_frequency=fs_empkins)
results_guardian_cleaned_ms[cols] = results_guardian_norm[cols] * convert_hz_to_ms(sampling_frequency=fs_guardian)

#### Concatenate the empkins and guardian dataframes

In [13]:
preprocessed_results_empkins_guardian = pd.concat([results_empkins_cleaned_ms, results_guardian_cleaned_ms])
preprocessed_results_empkins_guardian

heartbeat_id  \
                                                                                                   estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                   
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep                0   
                                                                                       Prep                1   
                                                                                       Prep                2   
                                                                                       Prep                3   
                                                                                       Prep                4   
...                                                                                                      ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     Dummy     TiltDown           51   
                                                                                       TiltDown           52   
                                                                                       TiltDown           53   
                                                                                       TiltDown           54   
                                                                                       TiltDown           55   

                                                                                                           \
                                                                                                reference   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep             0   
                                                                                       Prep             1   
                                                                                       Prep             2   
                                                                                       Prep             3   
                                                                                       Prep             4   
...                                                                                                   ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     Dummy     TiltDown        51   
                                                                                       TiltDown        52   
                                                                                       TiltDown        53   
                                                                                       TiltDown        54   
                                                                                       TiltDown        55   

                                                                                                heartbeat_start_sample  \
                                                                                                             estimated   
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                             
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep                      394.0   
                                                                                       Prep                      984.0   
                                                                                       Prep                     1569.0   
                                                                                       Prep                     2134.0   
                                                                                       Prep                     2684.0   
...                     

In [14]:
input_data = preprocessed_results_empkins_guardian.reset_index().set_index(['q_wave_algorithm', 'b_point_algorithm', 'outlier_correction_algorithm', 'participant', 'condition', 'phase'])

#### Select columns

In [15]:
if include_rr_interval:
    input_data = input_data[[("heartbeat_id", "reference"), ("q_wave_onset_sample", "estimated"), ("q_wave_onset_sample", "reference"), ("rr_interval_ms", "estimated")]]
else:
    input_data = input_data[[("heartbeat_id", "reference"), ("q_wave_onset_sample", "estimated"), ("q_wave_onset_sample", "reference")]]
input_data.columns = input_data.columns.to_flat_index().str.join("_")
input_data

heartbeat_id_reference  \
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                              
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep                           0   
                                                                                       Prep                           1   
                                                                                       Prep                           2   
                                                                                       Prep                           3   
                                                                                       Prep                           4   
...                                                                                                                 ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     Dummy     TiltDown                      51   
                                                                                       TiltDown                      52   
                                                                                       TiltDown                      53   
                                                                                       TiltDown                      54   
                                                                                       TiltDown                      55   

                                                                                                 q_wave_onset_sample_estimated  \
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                                     
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep                              187.0   
                                                                                       Prep                              186.0   
                                                                                       Prep                              182.0   
                                                                                       Prep                              174.0   
                                                                                       Prep                              170.0   
...                                                                                                                        ...   
vanlien2013-42-ms b-point-reference none                         GDN0030     Dummy     TiltDown                          338.0   
                                                                                       TiltDown                          336.0   
                                                                                       TiltDown                          346.0   
                                                                                       TiltDown                          324.0   
                                                                                       TiltDown                          316.0   

                                                                                                 q_wave_onset_sample_reference  
q_wave_algorithm  b_point_algorithm outlier_correction_algorithm participant condition phase                                    
forounzafar2018   b-point-reference none                         VP_001      tsst      Prep                              173.0  
                                                                                       Prep                              172.0  
                                                                                       Prep                              166.0  
                                                                                       Prep                              157.0  
                                

In [16]:
print(f"Amount of algorithms: {len(input_data.index.get_level_values("q_wave_algorithm").unique())}")

Amount of algorithms: 9


#### Transform dataframe from long to wide format

In [17]:
input_data = input_data.reset_index()
# Pivot the DataFrame
if include_rr_interval:
    input_data_wide = input_data.pivot(
        index=['participant', 'condition', 'phase', 'heartbeat_id_reference', 'q_wave_onset_sample_reference', 'rr_interval_ms_estimated'],
        columns="q_wave_algorithm",
        values='q_wave_onset_sample_estimated'
    ).reset_index()
else:
    input_data_wide = input_data.pivot(
        index=['participant', 'condition', 'phase', 'heartbeat_id_reference', 'q_wave_onset_sample_reference'],
        columns="q_wave_algorithm",
        values='q_wave_onset_sample_estimated'
    ).reset_index()

input_data_wide

q_wave_algorithm participant condition          phase  heartbeat_id_reference  \
0                    GDN0005     Dummy  HoldingBreath                       0   
1                    GDN0005     Dummy  HoldingBreath                       1   
2                    GDN0005     Dummy  HoldingBreath                       3   
3                    GDN0005     Dummy  HoldingBreath                       4   
4                    GDN0005     Dummy  HoldingBreath                       5   
...                      ...       ...            ...                     ...   
11606                 VP_032      tsst           Talk                      39   
11607                 VP_032      tsst           Talk                      40   
11608                 VP_032      tsst           Talk                      41   
11609                 VP_032      tsst           Talk                      42   
11610                 VP_032      tsst           Talk                      43   

q_wave_algorithm  q_wave_onset_sample_reference  forounzafar2018  \
0                                         260.0            276.0   
1                                         260.0            278.0   
2                                         222.0            236.0   
3                                         220.0            236.0   
4                                         228.0            242.0   
...                                         ...              ...   
11606                                     172.0            183.0   
11607                                     181.0            193.0   
11608                                     200.0            210.0   
11609                                     212.0            223.0   
11610                                     207.0            217.0   

q_wave_algorithm  martinez2004  scipy-findpeaks  vanlien2013-32-ms  \
0                        262.0            262.0              266.0   
1                        262.0            262.0              266.0   
2                        224.0            224.0              230.0   
3                        222.0            222.0              230.0   
4                        230.0            230.0              236.0   
...                        ...              ...                ...   
11606                    173.0            173.0              177.0   
11607                    182.0            182.0              185.0   
11608                    201.0            201.0              206.0   
11609                    213.0            213.0              220.0   
11610                      NaN              NaN              212.0   

q_wave_algorithm  vanlien2013-34-ms  vanlien2013-36-ms  vanlien2013-38-ms  \
0                             264.0              262.0              260.0   
1                             264.0              262.0              260.0   
2                             228.0              226.0              224.0   
3                             228.0              226.0              224.0   
4                             234.0              232.0              230.0   
...                             ...                ...                ...   
11606                         175.0              173.0              171.0   
11607                         183.0              181.0              179.0   
11608                         204.0              202.0              200.0   
11609                         218.0              216.0              214.0   
11610                         210.0              208.0              206.0   

q_wave_algorithm  vanlien2013-40-ms  vanlien2013-42-ms  
0                             258.0              256.0  
1                             258.0              256.0  
2                             222.0              220.0  
3                             222.0              220.0  
4                             228.0              226.0  
...                             ...                ...  
11606                         169.0          

#### Drop nan values and the 'scipy-findpeaks' column

In [19]:
input_data_wide_cleaned = input_data_wide.dropna()
input_data_wide_cleaned = input_data_wide_cleaned.drop(columns=["scipy-findpeaks"])
input_data_wide_cleaned

q_wave_algorithm participant condition          phase  heartbeat_id_reference  \
0                    GDN0005     Dummy  HoldingBreath                       0   
1                    GDN0005     Dummy  HoldingBreath                       1   
2                    GDN0005     Dummy  HoldingBreath                       3   
3                    GDN0005     Dummy  HoldingBreath                       4   
4                    GDN0005     Dummy  HoldingBreath                       5   
...                      ...       ...            ...                     ...   
11605                 VP_032      tsst           Talk                      38   
11606                 VP_032      tsst           Talk                      39   
11607                 VP_032      tsst           Talk                      40   
11608                 VP_032      tsst           Talk                      41   
11609                 VP_032      tsst           Talk                      42   

q_wave_algorithm  q_wave_onset_sample_reference  forounzafar2018  \
0                                         260.0            276.0   
1                                         260.0            278.0   
2                                         222.0            236.0   
3                                         220.0            236.0   
4                                         228.0            242.0   
...                                         ...              ...   
11605                                     153.0            188.0   
11606                                     172.0            183.0   
11607                                     181.0            193.0   
11608                                     200.0            210.0   
11609                                     212.0            223.0   

q_wave_algorithm  martinez2004  vanlien2013-32-ms  vanlien2013-34-ms  \
0                        262.0              266.0              264.0   
1                        262.0              266.0              264.0   
2                        224.0              230.0              228.0   
3                        222.0              230.0              228.0   
4                        230.0              236.0              234.0   
...                        ...                ...                ...   
11605                    154.0              179.0              177.0   
11606                    173.0              177.0              175.0   
11607                    182.0              185.0              183.0   
11608                    201.0              206.0              204.0   
11609                    213.0              220.0              218.0   

q_wave_algorithm  vanlien2013-36-ms  vanlien2013-38-ms  vanlien2013-40-ms  \
0                             262.0              260.0              258.0   
1                             262.0              260.0              258.0   
2                             226.0              224.0              222.0   
3                             226.0              224.0              222.0   
4                             232.0              230.0              228.0   
...                             ...                ...                ...   
11605                         175.0              173.0              171.0   
11606                         173.0              171.0              169.0   
11607                         181.0              179.0              177.0   
11608                         202.0              200.0              198.0   
11609                         216.0              214.0              212.0   

q_wave_algorithm  vanlien2013-42-ms  
0                             256.0  
1                             256.0  
2                             220.0  
3                             220.0  
4                             226.0  
...                             ...  
11605                         169.0  
11606                         167.0  
11607                         175.0  
11608                         196.0  
11

#### Save the preprocessed data to use it for training

In [22]:
print(f"Save results: {save_results}")
print(f"Include RR-Interval: {include_rr_interval}")

Save results: True
Include RR-Interval: False


In [21]:
if save_results:
    input_data_wide.to_csv(result_path.joinpath("data/q-peak/without-rr-interval/train_data_q_peak.csv"))
    print("Data saved!")

Data saved!
